In [2]:
import stable_baselines
from stable_baselines import A2C

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/Users/akumaar/opt/anaconda3/envs/tf/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [3]:
import numpy as np
import pandas as pd

In [6]:
X4 = pd.read_csv('/Users/akumaar/Downloads/X4_temp.csv')
prices = pd.read_csv('/Users/akumaar/Downloads/prices_from_2013.csv')

In [7]:
pool_prices = pd.merge(X4, prices, on = 'stock_id')

In [8]:
pool_prices['quarter'] = pd.to_datetime(pool_prices['quarter'])
pool_prices['date'] = pd.to_datetime(pool_prices['date']).dt.tz_localize(None)

In [9]:
pool_prices = pool_prices[pool_prices['date'] == pool_prices['quarter']]

In [10]:
pool_prices['numShares'] = np.zeros(8728)

In [11]:
quarter_groups = pool_prices.groupby(pool_prices.quarter)

In [12]:
test_data_20130930 = quarter_groups.get_group('2013-09-30')
test_data_20130930= test_data_20130930.drop(['Unnamed: 0', 'quarter','exchange','ticker','valume','open',
                                            'high','low'], axis = 1)

In [13]:
pool_prices = pool_prices.drop(['Unnamed: 0', 'quarter','exchange','ticker','valume','open',
                                            'high','low'], axis = 1)

In [15]:
pool_prices.to_csv('/Users/akumaar/Downloads/test_data_all_quarters.csv')

In [66]:
test_data_20130930.to_csv('/Users/akumaar/Downloads/test_data_2013-09-30.csv')

In [16]:
import gym
from gym.utils import seeding
from gym import spaces

In [17]:
Account_balance = 100000000
STOCK_DIM = len(np.unique(test_data_20130930['stock_id']))
HMAX_NORMALIZE = 100
REWARD_SCALING = 1e-4

##Changed boxes to Discrete
class environment(gym.Env):
    metadata = {'render.modes': ['human']}
    def __init__(self, df):
        self.df = df
        self.action_space = spaces.Box(low = -1, high = 1, shape = (STOCK_DIM,))
        self.observation_space = spaces.Box(low = 0, high = np.inf, shape = (1366,))
        
        self.state = [Account_balance] + \
                      self.df.close.values.tolist() + \
                      self.df.numShares.values.tolist() + \
                      self.df.percent_good_subfund.values.tolist()
                      
         
        self.reward = 0
        self.cost = 0
        self.asset_memory = Account_balance
        self.rewards_memory = []
        self.trades = 0
        
    def buy_shares(self, index, action):
        available_amount = self.state[0] // self.state[index+1]
        self.state[0] -= self.state[index+1]*min(available_amount, action)
        self.state[index+STOCK_DIM+1] += min(available_amount, action)

        self.cost+=self.state[index+1]*min(available_amount, action)
        self.trades+=1
    
    def sell_shares(self, index, action):
        if self.state[index+STOCK_DIM+1] > 0:
            #update balance
            self.state[0] += \
            self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) 

            self.state[index+STOCK_DIM+1] -= min(abs(action), self.state[index+STOCK_DIM+1])
            self.cost +=self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) 
            self.trades+=1
        else:
            pass

    def step(self, actions):
        actions = actions * HMAX_NORMALIZE
        print("after normalize")
        begin_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
        argsort_actions = np.argsort(actions)
        print("after argsort")
        sell_index = argsort_actions[:np.where(actions < 0)[0].shape[0]]
        buy_index = argsort_actions[::-1][:np.where(actions > 0)[0].shape[0]]
        
        for index in sell_index:
            print('take sell action'.format(actions[index]))
            self.sell_shares(index, actions[index])

        for index in buy_index:
            print('take buy action: {}'.format(actions[index]))
            self.buy_shares(index, actions[index])
            
        self.state = [self.state[0]] + \
                      self.df.close.values.tolist() + \
                      list(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]) + \
                      self.df.percent_good_subfund.values.tolist()
        
        end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
        self.asset_memory.append(end_total_asset)
            #print("end_total_asset:{}".format(end_total_asset))
        
        self.reward = end_total_asset - begin_total_asset            
        print("step_reward:{}".format(self.reward))
        self.rewards_memory.append(self.reward)
            
        self.reward = self.reward*REWARD_SCALING


        return self.state, self.reward
        
    def reset(self):

        self.asset_memory = [Account_balance]
        self.cost = 0
        self.trades = 0
        self.rewards_memory = []

        self.state = [Account_balance] + \
                      self.df.close.values.tolist() + \
                      self.df.numShares.values.tolist() + \
                      self.df.percent_good_subfund.values.tolist()
   
        return self.state
    def render(self, mode='human'):
        return self.state

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

In [58]:
from stable_baselines import A2C

import tensorflow
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy
from stable_baselines.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines.common.vec_env import DummyVecEnv
import time

def train_A2C(env_train, timesteps=10000):
    start = time.time()
    model = A2C('MlpPolicy', env_train, verbose=0)
    model.learn(total_timesteps=timesteps)
    end = time.time()
    print('Training time (A2C): ', (end-start)/60,' minutes')
    return model

In [65]:
env = environment(test_data_20130930)

TypeError: __init__() got an unexpected keyword argument 'low'

In [32]:
env.action_space()

array([-6.79636896e-01,  5.53040683e-01,  7.35940754e-01, -9.62875262e-02,
       -3.51778120e-01, -4.21762198e-01,  3.44321758e-01,  6.25689089e-01,
       -7.17475951e-01, -3.22951563e-02,  6.94589794e-01,  2.20628574e-01,
       -4.27883089e-01,  4.05985057e-01,  8.66153419e-01,  8.95253047e-02,
        5.09081632e-02,  4.87406403e-01,  4.38324720e-01,  5.33310294e-01,
        6.61256552e-01,  4.90308225e-01, -3.48570704e-01,  3.80908877e-01,
       -7.45093703e-01,  6.67651355e-01, -8.47925007e-01,  8.00481141e-02,
        6.42513275e-01,  6.44169390e-01, -1.36483207e-01, -4.27969456e-01,
       -8.83864537e-02, -7.09871590e-01,  6.73483238e-02,  1.24645531e-01,
        4.90242273e-01, -2.78575480e-01, -3.63998115e-01, -4.46263790e-01,
        6.06292665e-01,  7.41646588e-01,  1.93034843e-01, -4.65135396e-01,
        4.29313891e-02,  5.44846475e-01, -5.02517998e-01,  7.41993070e-01,
       -5.47095180e-01, -8.72904778e-01, -9.06685889e-01,  8.37519467e-01,
        2.45857850e-01,  

In [59]:
train_A2C(env)

resetting shii
after normalize
after argsort
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
take sell action
tak

ValueError: not enough values to unpack (expected 4, got 2)

In [62]:
q_table = np.zeros([env.observation_space.shape, env.action_space.shape])

TypeError: 'tuple' object cannot be interpreted as an integer

In [63]:
env.observation_space.shape

(1366,)